# I200455, Sameer Tahir, Assignment 4

In [2]:
import pandas as pd
import numpy as np
import random
import copy

In [3]:
data = pd.read_csv("Iris.csv")

In [4]:
data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
data.drop(columns=['Id'],inplace=True)

### Mapping Species

In [6]:
unique = data['Species'].unique()

dic= {}

for i,v in enumerate(unique):
    dic[v]=i

data['Species'] = data['Species'].map(dic)

In [7]:
data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


## Spliting Dataset

In [8]:
def Split(data,ratio):
    
    np.random.shuffle(data)
    
    X = data[:,:-1]
    Y = data[:,-1]
    
    samples = int(data.shape[0]*ratio)
   
    
    X_train = X[:samples,:]
    
    X_test = X[samples:,:]
    
    Y_train = Y[:samples]
    Y_test = Y[samples:]
            
    return (X_train,X_test,Y_train,Y_test)

In [9]:
X_train,X_test,Y_train,Y_test = Split(data.to_numpy(),0.7)

In [40]:
Y_test = Y_test.reshape(-1,1)

In [41]:
X_train.shape

(105, 4)

## Neural Network Model

In [72]:
class NeuralNetwork:
    
    def __init__(self, X_train, Y_train, hidden_layers, lst_nodes, lr, epochs):
        
        self.hidden_layers = hidden_layers
        self.lr=lr
        self.epochs = epochs
        self.lst_nodes = lst_nodes.copy()
        self.X_train = X_train
        self.Y_train = Y_train.reshape(-1,1).astype(int)
        
        
        input_nodes = X_train.shape[1]
#         print("Input Nodes:", input_nodes)
        
        
        # Initlised Weights and Bias
        self.parameters = self.initilise_parameters()
        
        for i in range(1,len(self.lst_nodes)):
            print( self.parameters[f'W{i}'].shape )
        
            

        self.train()
        
        
    
    
    def initilise_parameters(self):
        parameters = {}
        
        for i in range(1,len(self.lst_nodes)):
            print(self.lst_nodes[i-1],self.lst_nodes[i])
            parameters[f'W{i}'] = np.random.uniform(low=0,high=1,size = (self.lst_nodes[i-1],self.lst_nodes[i]))
            parameters[f'b{i}'] = np.random.uniform(low=0,high=1,size = (1,self.lst_nodes[i]))
                
            
            
        return parameters
                                                          
     
    def train(self):
        
            
        # Training
        for i in range(self.epochs):
            #forward propagation
            final_output,logits_dic = self.forward_pass()
            
            #loss
            loss,derv = self.loss_function(final_output ,self.Y_train)
            
            if(i%100 ==0):
                print(f"Loss after iteration {i} {loss}")
            
            #backward pass
            gradients = self.backward_pass(final_output,logits_dic)
            
            #update parameters
            self.gradient_decent(gradients)
        
        

        
        
      # Z is X*W + B
        # A is Relu(Z)
    def forward_pass(self):
        
        cache = {}
        
        cache['A0'] = self.X_train.copy()
        
        
        for i in range(1,len(self.lst_nodes)-1):
            
            # calculate Z
            cache[f'Z{i}'] = np.dot(cache[f'A{i-1}'], self.parameters[f'W{i}']) + self.parameters[f'b{i}']
            #calculate A
            cache[f'A{i}'] = self.Relu(cache[f'Z{i}'])
            print("halo")
            
            
        cache[f'Z{ len(self.lst_nodes)-1 }'] = np.dot(cache[f'A{ len(self.lst_nodes)-2 }'], self.parameters[f'W{len(self.lst_nodes)-1}']) + self.parameters[f'b{len(self.lst_nodes)-1}']
        cache[f'A{len(self.lst_nodes)-1}']  = cache[f'Z{ len(self.lst_nodes)-1 }'].copy()  
            
        return (cache[f'A{len(self.lst_nodes)-1}'],cache)
    
    
    def backward_pass(self,last,logits_dic):
        
        gradient = {}
        L = len(self.lst_nodes)-1
        m=self.Y_train.shape[0]
        
        gradient[f"dZ{L}"] = last - self.Y_train
        gradient[f"dW{L}"] = np.dot(logits_dic[f"A{L-1}"].T , gradient[f"dZ{L}"])
        gradient[f"db{L}"] = np.sum( gradient[f"dZ{L}"], axis=1, keepdims = True )
        
        
        
        for i in reversed(range(1,L)):
            print(f"Inside back loop{i}",np.dot(gradient[f'dZ{i+1}'], self.parameters[f"W{i+1}"].T).shape )
            
            gradient[f"dZ{i}"] = np.dot(gradient[f'dZ{i+1}'] ,self.parameters[f"W{i+1}"].T) * self.derivative_relu(logits_dic[f"A{i}"])
            gradient[f"dW{i}"] = np.dot(logits_dic[f"A{i}"].T , gradient[f"dZ{i}"])
            gradient[f"db{i}"] = np.sum( gradient[f"dZ{i}"], axis=1, keepdims = True )
        
        
        return gradient
        
        
    
    
    def loss_function(self , logits , y):
        
        diff = logits - np.array([logits[np.arange(y.shape[0]) , y.flatten()]]).T
        
        diff[diff < 0] = 0
        loss = np.sum(diff)
        
        diff = np.squeeze(diff)
        # Derivative
        diff[diff > 0] = 1
        diff[np.arange(y.shape[0]) , y] = -1 * np.sum(diff , axis = 1)
        return (loss, diff)
    
    
    def gradient_decent(self,gradients):
        old_parameters = self.parameters.copy()
        
        for i in range(1,len(self.lst_nodes)+1):
            self.parameters[f'W{i}'] = old_parameters[f'W{i}'] - self.lr * gradients[f'dW{i}']
            self.parameters[f'b{i}'] = old_parameters[f'b{i}'] - self.lr * gradients[f'db{i}']
            
    
    def Relu(self,value):
            return np.maximum(0, value)
    
    def derivative_relu(self,x):
        return np.array(x>0 , dtype = 'float')
    
    def get_parameters(self):
        return self.parameters
    

In [71]:
nn = NeuralNetwork(X_train,Y_train,4,[X_train.shape[1] ,5,2, 3],0.0001,1000)

4 5
5 2
2 3
(4, 5)
(5, 2)
(2, 3)
halo
halo
Inside back loop2 (105, 2)
Inside back loop1 (105, 5)
(5, 5)
(2, 2)
(2, 3)


In [36]:
Y_train

array([0., 1., 1., 0., 0., 1., 0., 1., 2., 0., 1., 0., 0., 2., 0., 0., 2.,
       0., 0., 2., 1., 1., 0., 2., 1., 2., 1., 2., 0., 0., 2., 2., 2., 0.,
       2., 0., 1., 0., 0., 0., 2., 1., 0., 0., 0., 2., 0., 1., 1., 2., 0.,
       1., 2., 2., 2., 0., 1., 2., 1., 2., 1., 1., 0., 0., 1., 2., 1., 1.,
       2., 2., 2., 2., 2., 2., 0., 1., 1., 0., 2., 1., 2., 0., 1., 1., 1.,
       2., 2., 2., 0., 1., 1., 2., 1., 0., 1., 1., 0., 0., 2., 0., 2., 1.,
       2., 1., 2.])